In [1]:
# import plotly.express as px
# import plotly.io as pio
# import plotly.graph_objects as go
# pio.renderers.default = "notebook_connected" # or use "browser" if you want plots to open with browser
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import einops
from fancy_einsum import einsum
from typing import List, Optional, Callable, Tuple, Union
import functools
from tqdm import tqdm
from IPython.display import display
import random
import os
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.utils import shuffle
import json

from transformer_lens.hook_points import HookPoint
from transformer_lens import utils, HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

# Saves computation time, since we don't need it for the contents of this notebook
t.set_grad_enabled(False)

In [2]:
gpt2_small = HookedTransformer.from_pretrained("gpt2-small")

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


In [3]:
dir(gpt2_small)

['OV',
 'QK',
 'T_destination',
 'W_E',
 'W_E_pos',
 'W_K',
 'W_O',
 'W_Q',
 'W_U',
 'W_V',
 'W_in',
 'W_out',
 'W_pos',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 

In [20]:
x = iter(gpt2_small.blocks[0].ln1.hook_scale.parameters())

In [21]:
next(x)

StopIteration: 

In [23]:
gpt2_small.blocks[0].ln2

LayerNormPre(
  (hook_scale): HookPoint()
  (hook_normalized): HookPoint()
)

In [24]:
gpt2_small.unembed

Unembed()

In [19]:
dir(gpt2_small.blocks[0].ln1.hook_scale.parameters())

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_suspended',
 'gi_yieldfrom',
 'send',
 'throw']

In [14]:
dir(gpt2_small.blocks[0].ln1.hook_scale)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook'

In [26]:
gpt2_small.ln_final

LayerNormPre(
  (hook_scale): HookPoint()
  (hook_normalized): HookPoint()
)

I think that in GPT2-small, all the layer norms are pre-folded (they only use the mean scaling + variance normalisation. Thanks Neel!)

In [27]:
gpt2_small

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_resid_pre): HookPoint()
      (hook_resid_mid): HookPoint()
   

In [32]:
gpt2_small.blocks[0].mlp.b_in

Parameter containing:
tensor([ 0.2700, -0.8441, -0.9387,  ..., -1.2354, -0.5161, -0.1925],
       requires_grad=True)

In [35]:
gpt2_small.blocks[0].mlp.W_in[:,0].shape

torch.Size([768])

In [38]:
gpt2_small.blocks[0].mlp.W_in[:,0]

tensor([ 1.0972e-02, -2.7837e-02,  8.8836e-03, -1.5953e-02, -5.1291e-02,
         6.3734e-02,  3.3048e-02, -5.8478e-04, -2.2426e-02,  2.2042e-02,
        -2.5507e-03, -3.3058e-03,  1.0504e-01, -5.4955e-02, -1.0614e-02,
        -7.5600e-03, -1.7999e-02,  1.1931e-01,  6.8784e-02, -5.3566e-02,
         6.4375e-02, -7.3302e-04,  1.5406e-01, -6.5432e-04,  1.7465e-01,
        -5.9878e-02, -1.5193e-02, -1.0737e-01,  8.3636e-02, -5.6585e-02,
        -1.9436e-02,  2.0272e-02, -7.3120e-02,  1.1758e-03, -1.0233e-01,
         2.4558e-02, -2.0279e-03, -8.6074e-02,  6.6153e-02, -2.4293e-02,
        -4.7778e-02, -6.4683e-02,  4.2292e-02,  1.3291e-01,  4.5853e-02,
        -8.9516e-02, -1.5959e-02, -8.3357e-02, -6.3525e-03, -1.3907e-02,
         6.1648e-02, -1.4339e-01, -3.0570e-03,  1.6816e-02,  2.6809e-02,
        -2.5912e-02,  7.4460e-02,  1.4382e-02, -3.2274e-02,  1.7539e-02,
        -9.6569e-03,  2.2307e-02, -6.0053e-02,  1.9773e-02, -4.2129e-03,
         2.4415e-02, -2.9629e-02,  2.5379e-02, -1.4

In [37]:
gpt2_small.blocks[0].mlp.W_in[:,1]

tensor([ 1.3067e-02, -1.3846e-02, -7.5037e-03,  1.9777e-01, -8.6101e-02,
         3.4741e-01, -5.3429e-03, -1.0845e-03, -6.2113e-02, -9.1335e-02,
         2.9534e-02,  1.5386e-01, -3.6974e-01,  2.4484e-01, -1.5712e-01,
         5.3124e-02, -1.2660e-02, -1.6937e-02, -5.6665e-02, -1.2858e-02,
         1.9126e-01, -7.3946e-02,  4.5079e-02, -6.1845e-03,  2.0435e-01,
        -7.5622e-02,  2.4018e-03,  1.6490e-02, -6.4517e-02,  3.4975e-01,
         5.7940e-02, -2.9955e-02, -1.5698e-01,  1.6101e-01, -4.3693e-02,
        -1.9378e-02, -6.4340e-03, -2.5251e-01, -1.3581e-01, -7.6953e-02,
        -2.7399e-01, -3.7754e-01,  3.4305e-01, -1.7915e-01,  2.7598e-02,
         3.3047e-01, -2.4168e-01,  1.8506e-01, -1.6470e-03,  1.6554e-02,
         1.8673e-02,  2.5340e-01,  9.4257e-03, -4.4691e-02,  6.5146e-03,
        -2.0624e-02, -2.3529e-01,  7.0626e-03, -4.1568e-02,  3.1047e-03,
         3.4290e-02, -3.1181e-01,  4.9924e-03,  2.9296e-02, -5.8942e-03,
        -1.6427e-01, -1.2766e-01, -2.6220e-02,  1.9